In [9]:

import pickle
import numpy as np
from numpy import array, ones,zeros
import sys


In [10]:
def insertion_viterbi2(c_tags,proba_inits,proba_tran,proba_obs,obs):
    L = len(obs)
    V = dict()
    psi = dict()
    # if p(y0)*p(x0|y0)*p(y1|y0)*p(x1|y1)<p(additional_y0)*p(y0|additional_y0)*p(x0|y0)*p(y1|y0)*p(x1|y1) ,we add a state
    for y in c_tags:
        
        V[0,y] = proba_inits[y] * proba_obs[(obs[0],y)]
        psi[0,y] = 0
    t = 1
    for t in range(1,L):
        for y in c_tags:
            (prob, state1) = max([(V[t-1,y0]* proba_tran[y0,y] * proba_obs[obs[t],y], y0) for y0 in c_tags])
            temp = []
            l = 0
            for x in c_tags:
                add_state_proba ,state= max( [(V[t-1,y0]* proba_tran[y0,x] * proba_tran[x,y] *proba_obs[obs[t],y] ,y0) for y0 in c_tags]) 
                temp.append((add_state_proba,x,state))
            l = max(temp)[0]
            k = (max(temp)[1],max(temp)[2])
            if l>prob:
                psi[t,y] = k
                V[t,y] = l
            else:
                psi[t,y] = state1
                V[t,y] = prob
    temp = []        
    for i in c_tags:
        temp.append((V[L-1,i],i))
    last = max(temp)[1]
    result = []
    result.append(last)
    for i in range(L-1,0,-1):
        temp_last = psi[i,result[-1]]
        if len(temp_last) == 2:
            result.append(temp_last[0])
            result.append(temp_last[1])
        else:
            result.append(temp_last)
    result.reverse()
    return result

In [11]:
with open('train10.pkl','rb') as f:
    data = pickle.load(f)
def make_counts(corpus):
    c_tags = dict()
    c_obs = dict()
    c_trans = dict()
    c_pairs = dict()
    c_inits = dict()
    for word in corpus:
        for i in range(len(word)):
            pair = word[i]
            tag = pair[1]
            obs = pair[0]
            if tag in c_tags:
                c_tags[tag]+=1
            else:
                c_tags[tag]=1
            if obs in c_obs:
                c_obs[obs]+=1
            else:
                c_obs[obs]=1
            if pair in c_pairs:
                c_pairs[pair]+=1
            else:
                c_pairs[pair]=1
            if i>0:
                trans = (word[i-1][1],tag)
                if trans in c_trans:
                    c_trans[trans]+=1
                else:
                    c_trans[trans]=1
            else:
                if tag in c_inits:
                    c_inits[tag]+=1
                else:
                    c_inits[tag]=1
    return c_obs, c_tags, c_pairs, c_trans, c_inits

In [12]:
c_obs, c_tags, c_pairs, c_trans, c_inits = make_counts(data)
s = 0
proba_inits = dict.fromkeys(c_tags)
for i in proba_inits:
    proba_inits[i] = 0
for i in c_inits:
    s = s + c_inits[i]
for i in c_inits:
    proba_inits[i] = c_inits[i]/s
from __future__ import division
proba_tran = dict()
s = dict()
for i in c_tags:
    s[(i)] = 0
    for j in c_tags:
        proba_tran[(i,j)] = 0
for i in c_trans:
    s[i[0]] = s [i[0]] + c_trans[i]
for i in c_trans:
    proba_tran[i] = c_trans[i]/s[i[0]]
proba_obs = dict()
for i in c_tags:
    for j in c_tags:
        proba_obs[(i,j)]=0
for i in c_pairs:
    proba_obs[i] = c_pairs[i]/c_tags[i[1]]

In [16]:
state1 = ['b','e','d']
obs1 = ['b','d']
state2 = ['w','o','u','l','d'] 
obs2 = ['w','u','l','d']
obs3 = ['p', 's', 'y', 'c', 'h', 'o', 'l', 'g', 'i', 'c', 'a', 'l']
state3 =  ['p', 's', 'y', 'c', 'h', 'o', 'l', 'o', 'g', 'i', 'c', 'a', 'l']
obs4 =  ['q', 'c', 't', 'v', 'i', 't', 'i', 'e', 's']
state4 = ['a', 'c', 't', 'i', 'v', 'i', 't', 'i', 'e', 's']
print "the observation is    " ,obs1
print "the return of our algorithm is      ",insertion_viterbi2(c_tags,proba_inits,proba_tran,proba_obs,obs1)
print "the actual state is",    state1

print "==================================================================================="

print "the observation is    ",obs2
print "the return of our algorithm is      ",insertion_viterbi2(c_tags,proba_inits,proba_tran,proba_obs,obs2)
print "the actual state is",state2

print "==================================================================================="

print "the observation is    ",obs3
print "the return of our algorithm is      ",insertion_viterbi2(c_tags,proba_inits,proba_tran,proba_obs,obs3)
print "the actual state is",state3

print "==================================================================================="

print "the observation is    ",obs4
print "the return of our algorithm is      ",insertion_viterbi2(c_tags,proba_inits,proba_tran,proba_obs,obs4)
print "the actual state is",state4

the observation is     ['b', 'd']
the return of our algorithm is       ['b', 'e', 'd']
the actual state is ['b', 'e', 'd']
the observation is     ['w', 'u', 'l', 'd']
the return of our algorithm is       ['w', 'o', 'u', 'l', 'd']
the actual state is ['w', 'o', 'u', 'l', 'd']
the observation is     ['p', 's', 'y', 'c', 'h', 'o', 'l', 'g', 'i', 'c', 'a', 'l']
the return of our algorithm is       ['p', 's', 'y', 'c', 'h', 'o', 'l', 'o', 'g', 'i', 'c', 'a', 'l']
the actual state is ['p', 's', 'y', 'c', 'h', 'o', 'l', 'o', 'g', 'i', 'c', 'a', 'l']
the observation is     ['q', 'c', 't', 'v', 'i', 't', 'i', 'e', 's']
the return of our algorithm is       ['a', 'c', 't', 'i', 'v', 'i', 't', 'i', 'e', 's']
the actual state is ['a', 'c', 't', 'i', 'v', 'i', 't', 'i', 'e', 's']
